In [1]:
'''

The words_before/after list would look like 

[['~'], '598'],
 ['-'], '143'],
 ... 
 

'''

with open('index_before.txt', encoding="utf8") as f:
    # before lower casing
    words_before = [line.split() for line in f.read().split('\n') if line]
    

with open('index_after.txt', encoding="utf8") as f:
    # after lower casing 
    words_after = [line.split() for line in f.read().split('\n') if line]

In [2]:
print(len(words_before)) # the length of words before lower casing - 52788
print(len(words_after)) # the length of words after lower casing  - 50966

52788
50966


In [3]:
#  to get the 10 most frequent words from the lowercased test collections
def stop_words(dictionary, top_n_words): 
    stop_words = []
    i = 0
    for k in sorted(dictionary, key=lambda k: len(dictionary[k]), reverse=True):
        if i < top_n_words:
            stop_words.append(k)
        i += 1
        
    return stop_words

def inverted_index(indexlist, stop_words_processing = False, query=None):
    vocab = {} #make new dict in format of {word: doc1, doc10, doc7    word2: doc 8, doc 19,    wordN: etc.}
    for i, word in enumerate(indexlist): #i = index, word is list of ['~', '598'] (words_after), indexlist = words_after
        if word[0] in vocab: #word[0] is the character/word/feature, check if already in vocab dict
            vocab.get(word[0]).append(word[1]) #if so, get the value of that word and append the new doc# to it
        else:
            vocab[word[0]] = [word[1]] #else, make a new dict entry with that word and set value = to doc#
            
    # stop_word processing if the 2nd parameter was given 'True'    
    if stop_words_processing == True:
        stopwords = stop_words(vocab, 10) #pass in your constructed vocab and desired top 10 most words
        for k, v in list(vocab.items()): #traverse vocab dict, k = words, v = doc ID #s
            if k in stopwords: #if vocab word in stopwords
                del vocab[k] #delete the vocab dict entry of that vocab word
   

    if query:
        for k, v in list(vocab.items()): #traverse vocab dict, k = words, v = doc ID #s
            if k == query: #if vocab word is the word you passed thru into function
                print(f"The doc ID for your query '{k}' is   {v}") #print word and word's value list of doc IDs
                return
    
    # sum of the length of the postings lists 
    sum_len_values = sum(len(dct) for dct in vocab.values())           
    return len(vocab), sum_len_values # len(vocab) for the size of the dictionary     

In [4]:
# before stop words processing 
print(inverted_index(words_after, False))

# after stop words processing 
print(inverted_index(words_after, True))

# simple query 
print(inverted_index(words_after, True, 'school'))

(3684, 50966)
(3674, 43163)
The doc ID for your query 'school' is   ['72', '111', '223', '224', '268', '385', '431', '494', '532', '553', '554', '564', '581', '582', '996']
None


In [19]:
'''
function that performs the intersection 
between two or more posting lists 
'''

def intersect(lst1, lst2):
    # e.g. str '2' -> int 2 
    lst1 = list(map(int, lst1))
    lst2 = list(map(int, lst2))
    
    res = [];
    i=0 ; j = 0; compare_count = 0;
    
    while i<len(lst1) and j<len(lst2):
        if lst1[i] == lst2[j]:
            res.append(lst1[i])
            i += 1
            j += 1
            compare_count += 1
        
        elif lst1[i] < lst2[j]:
            i += 1
            compare_count += 1 
        
        elif lst1[i] > lst2[j]:
            j += 1
            compare_count += 1 

            
    print("No.of comparison: ", compare_count)
            
    return res

    
def intersect_w_skip(lst1, lst2):
    skip_size = 3
    has_skip = lambda x: x % skip_size == 0
    
    lst1 = list(map(int, lst1))
    lst2 = list(map(int, lst2))
    
    res = [];
    i=0 ; j = 0; compare_count = 0;
    
    while i<len(lst1) and j<len(lst2):
        
        if lst1[i] == lst2[j]:
            res.append(lst1[i])
            i += 1
            j += 1
            compare_count += 1 
            
            
        elif has_skip(i) and i + skip_size < len(lst1) and lst1[i+skip_size] < lst2[j]:
            i += skip_size 
            compare_count += 1
        
        elif lst1[i] < lst2[j]:
            i += 1
            compare_count += 1
        
        elif has_skip(j) and j + skip_size < len(lst2) and lst2[j+skip_size] < lst1[i]:
            j += skip_size
            compare_count += 1
        
        elif lst1[i] > lst2[j]:
            j += 1 
            compare_count += 1
           
    print("No.of comparison: ", compare_count)
            
    return res

def query(indexlist):
    vocab = {}
    for i, word in enumerate(indexlist):
        if word[0] in vocab:
            vocab.get(word[0]).append(word[1])       
        else:
            vocab[word[0]] = [word[1]]
            
    stopwords = stop_words(vocab, 10)
    
    for k, v in list(vocab.items()):
        if k in stopwords:
            del vocab[k]    
    
    lst = []; 
    
    user_input = input("Enter an intersection query (format: a AND b): ") #ask user input
    user_input = user_input.lower() #to account for user not capitalizing 'AND' condition
    input_list = user_input.split(' and ') #split user input into list around the 'and' condition. result is list of words they want to intersect
    
    for word in input_list: #check if all user input words are in the vocab
        if word not in vocab:
            print(f"Query entry '{word}' not in vocabulary. Try again.")
            return
    
    for i, query in enumerate(input_list): #traverse thru user list, i =index of user list, query = word user asking for
        for k, v in list(vocab.items()): #traverse thru vocab words, k = vocab word, v = doc IDs
            if k == query: #if vocab word == word user wants
                lst.append(v) #append doc IDs into list
                
    
                
    sorted_list = sorted(lst, key=len) #sort by the length of the # of docs each word has
    res = intersect_w_skip(sorted_list[0], sorted_list[1]) #get interest of both words
    
    if len(lst) == 2: #if only 2 entries for comparison
        return res #return intersection

    elif len(lst) > 2: #more than 2 words
        k = 2
        while k < len(sorted_list):
            res = intersect_w_skip(res, sorted_list[k]) #compare the first 2 words' intersection with the next one until you're at the max words
            k += 1 #increment k
            
        return res #return intersection
            

In [20]:
# school AND kids AND really
query(words_after)

Enter an intersection query (format: a AND b):  school and kids and really


No.of comparison:  28
No.of comparison:  35


[72, 224, 385]

In [7]:
# Ranked queries for the query 'school AND kids AND really' 

with open('index_rank.txt', encoding = 'utf8') as f:
    # after lower casing 
    words_after_rank = [line.split() for line in f.read().split('\n') if line]
    
for i in words_after_rank:
    i[0], i[2] = int(i[0]), int(i[2]) 

In [8]:
# inverted document frequency 
print(inverted_index(words_after, True, 'school'))
print(inverted_index(words_after, True, 'kids'))
print(inverted_index(words_after, True, 'really'))

The doc ID for your query 'school' is   ['72', '111', '223', '224', '268', '385', '431', '494', '532', '553', '554', '564', '581', '582', '996']
None
The doc ID for your query 'kids' is   ['32', '72', '175', '224', '336', '385', '459', '460', '475', '523', '538', '539', '553', '583', '584', '593', '613', '786', '806', '970']
None
The doc ID for your query 'really' is   ['6', '16', '31', '36', '38', '40', '44', '45', '51', '54', '56', '58', '71', '72', '73', '77', '78', '84', '91', '99', '112', '125', '133', '139', '151', '173', '179', '182', '184', '189', '190', '198', '202', '205', '223', '224', '225', '231', '238', '246', '256', '270', '285', '300', '312', '334', '340', '342', '344', '349', '350', '358', '361', '365', '368', '385', '387', '392', '401', '409', '418', '433', '447', '456', '475', '480', '482', '489', '494', '499', '503', '505', '507', '508', '515', '516', '523', '525', '526', '546', '570', '572', '573', '584', '596', '598', '601', '610', '619', '647', '649', '683', '691

# Compute the TF/IDF values for the query terms school, kids and really for the documents that contain all three terms.

In [9]:
tf = {72: {'school': 0, 'kids': 0, 'really': 0},
      224: {'school': 0, 'kids': 0, 'really': 0}, 
      385: {'school': 0, 'kids': 0, 'really': 0}}
for i in words_after_rank:
    if i[1] == 'school' or i[1] == 'kids' or i[1] == 'really':
        if i[2] == 72 or i[2] == 224 or i[2] == 385:
            tf[i[2]][i[1]] += 1
               
import math 

N = len(tf) # total num of documents
idf = math.log(4/4+1)

tfidf = tf
for doc, word in tfidf.items():
    for num, value in word.items():
        word[num] *= idf

In [25]:
tf

{72: {'school': 2, 'kids': 1, 'really': 1},
 224: {'school': 1, 'kids': 1, 'really': 1},
 385: {'school': 2, 'kids': 1, 'really': 3}}

In [10]:
tfidf

{72: {'school': 1.3862943611198906,
  'kids': 0.6931471805599453,
  'really': 0.6931471805599453},
 224: {'school': 0.6931471805599453,
  'kids': 0.6931471805599453,
  'really': 0.6931471805599453},
 385: {'school': 1.3862943611198906,
  'kids': 0.6931471805599453,
  'really': 2.0794415416798357}}

In [11]:
idf

0.6931471805599453

# Lab Report

__1. How to run our code__  

We've commented our code along the way for your (the grader's) convenience, and for us to review. We started by importing our data after completing the command line commands, leaving us with index files of before and after lower casing the data. We left a printout of the number of words before and after doing this for you to review. We had a decrease in 1,822 words. To get the top k (10) – most frequent words (from the lowerized test data, we traversed through the passed through data and sorted it by the length of its frequency in the dictionary (our variable name) and kept appending each word into a stop words list until our counter reached the desired top-k words. 

__2. Inverted Index / Boolean queries__

For our inverted index function, we left very descriptive commenting to act as our explanation for that part. We then left a printout of how every possible outcome of the inverted index function would be depending on the parameters desired by the user. Next, our intersect functions we passed in 2 lists of documents containing document ID numbers of each word the user inputted to compare. We implemented the main chunk of this (and the intersect with skips) function using the ‘Boolean retrieval’ lecture from the professor. While the index of both lists being compared isn’t NoneType (inducing an Index Error), we see if the operator comparison between the two elements of the lists at 2 equal, but distinct variables, is equal to (==) each other, if list1[index1] is less than (<) list2[index2], or if list1[index1] is greater than (>) list2[index2]. Each of these comparisons ill trigger the respective actions: appendage of current list1[index1] onto a list to be returned later, increment of both index1 and index2, and a comparison counter, or an increment of index1 and the comparison counter, or an increment of index2 and comparison counter. And at the end of this function, we print out the number of comparisons to be viewed and the resultant list we’ve been appending to, which is the intersect of document IDs of the user’s words. Intersect with skip has the same structure, but we’ve self-defined our skip size as 3 for experimental purposes, which you can change if you feel inclined. We then created a lambda function to act as our condition later on. There are extra simple if-statement conditions to check if the current index has a skip, if that index plus (+) the skip size is still less than (<) the length of the list (this is to make sure you don’t cause an out of bounds indexing error), and that the current list’s index at the current index plus skip size is still less than list2 at its current index. And the second new conditional statement is functionally the same, but from list2’s perspective. And again, we print out the comparisons and return the intersects. For our query we left commentary for you. We also left an example printed out. The number of comparisons when using the skip functions makes a sizeable difference, that would be beneficial in this query. When searching for the intersection of 'school AND kids AND really', using the skip counter with size 2 gives 71 comparisons, without it it gives 111 comparisons. With a skip size of 3, it gives 63 comparisons with the skip counter.

__3. Ranked queries__ 

We then moved on to our index ranking to complete the rest of the assignment. To calculate the TF-IDF we created a nested dictionary for the intersection of document IDs 72, 224, and 385, which represented the intersection of the keywords ‘school’, ‘kids’ and ‘really’. The inner values were initialized at 0 and we traversed our words after ranking and if the corresponding indices match the one of the previously mentioned words, and the document IDs, then we would increment the appropriate inner values. We then compute idf with the log method from the math library, and multiply the idf by each tf to get our final answer. 

Our results were clear that ‘school’ was the most “important” word in this particular list of words to compare to. It can be inherently understood why because the word ‘school’ would most likely be involved in documents with ‘kids’. However, in one particular document, 385, the word ‘really’ appears relatively “much” more than in the other documents and so for this particular document/case, it bears with it more importance than ‘school’, yielding in a higher TF-IDF value, 2.07945.
